In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from tqdm import tqdm
import akshare as ak
import datetime
from multiprocessing import Pool
from sklearn.metrics import confusion_matrix

In [2]:
def date_strategy(stockcode, show=0):
    # 导入股票
    try:
        stock_code = ak.stock_zh_a_hist(symbol=stockcode, period="daily", start_date='20200101', end_date='20250306', adjust="hfq")[['日期','股票代码','开盘','收盘','最高','最低','成交量','成交额','换手率']]
    except:
        return stockcode, np.nan, np.nan, np.nan, np.nan
    if len(stock_code)<100:
        return stockcode, np.nan, np.nan, np.nan, np.nan
    if (stock_code['收盘'] < 0).any():
        return stockcode, np.nan, np.nan, np.nan, np.nan

    stock_code.rename(columns={'日期':'date','股票代码':'stockcode','成交量':'turnovervolumn','成交额':'turmovervalue','换手率':'turnoverrate',
                                '开盘':'backward_openprice','收盘':'backward_closeprice','最高':'backward_highprice','最低':'backward_lowprice'},inplace=True)
    stock_code['date'] = stock_code['date'].astype(str)
    stock_code.set_index(['date','stockcode'],inplace=True)
    stock_code.sort_index(inplace=True)
    stock_code_beifen = stock_code.copy()

    # 指标计算
    stock_code['MA_5']  = stock_code['backward_closeprice'].rolling(window=5,min_periods=5).mean()
    stock_code['MA_10'] = stock_code['backward_closeprice'].rolling(window=10,min_periods=10).mean()
    stock_code['MA_20'] = stock_code['backward_closeprice'].rolling(window=20,min_periods=20).mean()
    stock_code['MA_60'] = stock_code['backward_closeprice'].rolling(window=60,min_periods=60).mean()
    stock_code['MA_90'] = stock_code['backward_closeprice'].rolling(window=90,min_periods=90).mean()

    stock_code['bias_5']  = stock_code['backward_closeprice'] - stock_code['MA_5']
    stock_code['bias_10'] = stock_code['backward_closeprice'] - stock_code['MA_10']
    stock_code['bias_20'] = stock_code['backward_closeprice'] - stock_code['MA_20']
    stock_code['bias_60'] = stock_code['backward_closeprice'] - stock_code['MA_60']
    stock_code['bias_90'] = stock_code['backward_closeprice'] - stock_code['MA_90']

    stock_code['std_5']  = stock_code['backward_closeprice'].rolling(window=5,min_periods=5).std()
    stock_code['std_10'] = stock_code['backward_closeprice'].rolling(window=10,min_periods=10).std()
    stock_code['std_20'] = stock_code['backward_closeprice'].rolling(window=20,min_periods=20).std()
    stock_code['std_60'] = stock_code['backward_closeprice'].rolling(window=60,min_periods=60).std()
    stock_code['std_90'] = stock_code['backward_closeprice'].rolling(window=90,min_periods=90).std()

    delta = stock_code['backward_closeprice'].diff(1)
    stock_code['RSI_10'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=10).mean()
    stock_code['RSI_20'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=20).mean()
    stock_code['RSI_60'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=60).mean()
    stock_code['RSI_90'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=90).mean()

    stock_code['VCV_10'] = stock_code['turnovervolumn'].rolling(10).std() / stock_code['turnovervolumn'].rolling(10).mean()
    stock_code['VCV_20'] = stock_code['turnovervolumn'].rolling(20).std() / stock_code['turnovervolumn'].rolling(20).mean()
    stock_code['VCV_60'] = stock_code['turnovervolumn'].rolling(60).std() / stock_code['turnovervolumn'].rolling(60).mean()
    stock_code['VCV_90'] = stock_code['turnovervolumn'].rolling(90).std() / stock_code['turnovervolumn'].rolling(90).mean()

    stock_code['VPT'] = (stock_code['turnovervolumn'] * (stock_code['backward_closeprice'].pct_change())).cumsum()
    stock_code['Price_Vol_Corr'] = stock_code['backward_closeprice'].pct_change().rolling(20).corr(stock_code['turnovervolumn'].pct_change())
    stock_code['OBV'] = (stock_code['turnovervolumn'] * stock_code['backward_closeprice'].pct_change().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))).cumsum()
    stock_code['VR_20'] = stock_code['turnovervolumn'].where(stock_code['backward_closeprice'].diff() > 0, 0).rolling(20).sum() / stock_code['turnovervolumn'].where(stock_code['backward_closeprice'].diff() < 0, 0).rolling(20).sum()
    stock_code['PAV_20'] = stock_code['turnovervolumn'] / stock_code['backward_closeprice'].rolling(20).mean()

    # 滚动标准化
    stock_code.fillna(stock_code.mean(),inplace=True)
    stock_code = (stock_code - stock_code.rolling(100).mean()) / stock_code.rolling(100).std()
    stock_code.fillna(stock_code.mean(),inplace=True)
    stock_code = stock_code.iloc[100:,:]

    # 打标签
    stock_code['raw_price'] = stock_code_beifen['backward_closeprice']
    stock_code['label'] = 0
    stock_code.loc[stock_code['raw_price'].shift(-5) > stock_code['raw_price'],'label'] = 1
    stock_code.drop(['raw_price'],axis=1,inplace=True)


    if len(stock_code)<130:
        return stockcode, np.nan, np.nan, np.nan, np.nan
    pred_list = []

    for i in range(0,len(stock_code),30):
        train_X = stock_code.iloc[i:100+i,:-1]
        train_y = stock_code.iloc[i:100+i,-1]
        
        if 135 + i >= len(stock_code):
            test_X = stock_code.iloc[105+i:(len(stock_code)+1),:-1]
            test_y = stock_code.iloc[105+i:(len(stock_code)+1),-1]

            model.fit(train_X, train_y)
            y_pred = model.predict(test_X)
            y_pred = pd.DataFrame({'pred':y_pred, 'label':test_y},index=test_y.index)
            pred_list.append(y_pred)
            break

        else:
            test_X = stock_code.iloc[105+i:135+i,:-1]
            test_y = stock_code.iloc[105+i:135+i,-1]

            model.fit(train_X, train_y)
            y_pred = model.predict(test_X)
            y_pred = pd.DataFrame({'pred':y_pred, 'label':test_y},index=test_y.index)
            pred_list.append(y_pred)

    pred_result = pd.concat(pred_list)
    pred_result['backward_closeprice'] = stock_code_beifen['backward_closeprice']


    # 策略回测
    fund = 1
    hold = 0
    fund_list = []
    hold_list = []
    date_list = []
    sign_list = []
    hold_day = 5

    for date in pred_result.index:
        if hold_day < 5:
            fund = hold * pred_result.loc[date,'backward_closeprice']
            hold = hold
            fund_list.append(fund)
            hold_list.append(hold)
            date_list.append(date[0])
            sign_list.append(0)
            hold_day += 1
            continue
        
        if pred_result.loc[date,'pred'] == 1 :

            if hold == 0:
                fund = fund
                hold = fund / pred_result.loc[date,'backward_closeprice']
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(1)
                hold_day = 0
                continue

            if hold != 0:
                fund = hold * pred_result.loc[date,'backward_closeprice']
                hold = hold
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(0)
                continue

        if pred_result.loc[date,'pred'] == 0 :
            if hold == 0:
                fund = fund
                hold = 0
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(0)
                continue

            if hold != 0:
                fund = hold * pred_result.loc[date,'backward_closeprice']
                hold = 0
                fund_list.append(fund)
                hold_list.append(hold)
                date_list.append(date[0])
                sign_list.append(-1)
                continue

    # 计算回测收益
    PnL = pd.DataFrame({'strategy':fund_list,'hold':hold_list,'sign':sign_list},  pd.MultiIndex.from_arrays([date_list, [stockcode] * len(date_list)],  names=['date', 'stock_code']))
    PnL['price'] = stock_code['backward_closeprice']
    PnL['price'] = PnL['price']/ PnL.loc[PnL.index[0],'price']

    # 计算回测指标
    returns = PnL['strategy'].pct_change().dropna()  # 计算每日收益率
    annual_return = (PnL['strategy'].iloc[-1] / PnL['strategy'].iloc[0]) ** (252 / len(PnL)) - 1
    annual_volatility = returns.std() * np.sqrt(252)
    cum_max = PnL['strategy'].cummax()  # 计算累计最大净值
    drawdown = PnL['strategy'] / cum_max - 1  # 计算回撤
    max_drawdown = drawdown.min()  # 取最大回撤
    sharpe_ratio = annual_return / annual_volatility if annual_volatility != 0 else np.nan

    if show == 1:
        # 策略结果可视化
        fig, ax1 = plt.subplots(figsize=(20, 6))
        df = PnL.copy(deep=True)
        df.index = pd.MultiIndex.from_tuples([(pd.to_datetime(date), stock) for date, stock in df.index], names=['date', 'stock_code'])

        # 折线图
        ax1.plot(df.index.get_level_values('date'), df['price'], label='price', color='tab:blue', linewidth=1)
        ax1.plot(df.index.get_level_values('date'), df['strategy'], label='strategy', color='tab:green', linestyle='dashed')

        # 添加买卖点
        buy_signals = df[df['sign'] == 1]
        sell_signals = df[df['sign'] == -1]
        ax1.scatter(buy_signals.index.get_level_values('date'), buy_signals['price'], color='red', label='buy', marker='o', s=15)
        ax1.scatter(sell_signals.index.get_level_values('date'), sell_signals['price'], color='blue', label='sell', marker='o', s=15)

        # 添加文本信息
        textstr = f"annual_return: {annual_return:.2%}\nannual_volatility: {annual_volatility:.2%}\nmax_drawdown: {max_drawdown:.2%}\nsharpe_ratio: {sharpe_ratio:.2f}"
        ax1.text(df.index.get_level_values('date')[-int(len(df) * 0.2)], df['price'].max() * 0.9, textstr,
                fontsize=12, bbox=dict(facecolor='white', alpha=0.6))

        ax1.set_xlabel('date')
        ax1.set_ylabel('net value')
        ax1.set_title(f'stockcode {PnL.index[0][1]} return')
        ax1.legend()
        plt.grid(alpha=0.5)
        plt.show()

    return stockcode,annual_return, annual_volatility, max_drawdown, sharpe_ratio,PnL,pred_result

In [3]:
# 获取股票列表
today_stock = ak.stock_zh_a_spot_em()[['代码']]
stock_code_list = sorted(today_stock['代码'].to_list())

# 载入模型
from sklearn.svm import SVC
model = SVC(kernel='rbf', C=1.0, gamma='scale', class_weight={0: 0.5, 1: 1})

# 并行运算策略
with Pool() as pool:
    results = list(tqdm(pool.imap(date_strategy, stock_code_list), total=len(stock_code_list)))

# 保存策略结果
stock_code_list = []
stock_annual_return = []    
stock_annual_volatility = []
stock_max_drawdown = []
stock_sharpe_ratio =[]
for data in results:
    stock_code_list.append(data[0])
    stock_annual_return.append(data[1])
    stock_annual_volatility.append(data[2])
    stock_max_drawdown.append(data[3])
    stock_sharpe_ratio.append(data[4])

100%|██████████| 5697/5697 [01:18<00:00, 72.89it/s] 


In [4]:
# 策略结果分析
strategy_result = pd.DataFrame(
    {
        'stockcode':stock_code_list,
        'annual_return':stock_annual_return,
        'annual_volatility':stock_annual_volatility,
        'max_drawdown':stock_max_drawdown,
        'sharpe_ratio':stock_sharpe_ratio,
    }
)
strategy_result.dropna(inplace=True)
strategy_result.set_index(['stockcode'],inplace=True)

In [5]:
strategy_result.mean() # 保守派

annual_return       -0.019286
annual_volatility    0.200956
max_drawdown        -0.348599
sharpe_ratio        -0.132956
dtype: float64

In [5]:
strategy_result.mean() # 正常派

annual_return       -0.014811
annual_volatility    0.317154
max_drawdown        -0.483287
sharpe_ratio        -0.086693
dtype: float64

In [5]:
strategy_result.mean() # 激进派

annual_return       -0.008518
annual_volatility    0.399482
max_drawdown        -0.552459
sharpe_ratio        -0.062681
dtype: float64